# ADS-500B Final Team Project

## Team: Stephanie Smith, Luisa Gonzalez, Zulfiqar Khan
### Dataset: Annual User sessions data for an e-commerce company

#### Data Importing and Pre-processing

In [39]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [224]:
# Load data
df = pd.read_csv('online_shoppers_intention.csv')

In [226]:
# Confirm file type is CSV
file_path = 'online_shoppers_intention.csv'
file_type= os.path.splitext(file_path)[1]
print(file_type)

# Confirm file was imported as a Pandas DataFrame
is_dataframe = isinstance(df, pd.DataFrame)
print(is_dataframe)


.csv
True


In [228]:
# Examine the first 10 rows of the dataset

df.head(10)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0.0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,Feb,1.0,1,1,1,Returning_Visitor,False,False
1,0,0.0,0.0,0.0,2,64.000000,0.000000,0.100000,0.0,0.0,Feb,2.0,2,1,2,Returning_Visitor,False,False
2,0,0.0,0.0,0.0,1,0.000000,0.200000,0.200000,0.0,0.0,Feb,4.0,1,9,3,Returning_Visitor,False,False
3,0,0.0,0.0,0.0,2,2.666667,0.050000,0.140000,0.0,0.0,Feb,3.0,2,2,4,Returning_Visitor,False,False
4,0,0.0,0.0,0.0,10,627.500000,0.020000,0.050000,0.0,0.0,Feb,3.0,3,1,4,Returning_Visitor,True,False
5,0,0.0,0.0,0.0,19,154.216667,0.015789,0.024561,0.0,0.0,Feb,2.0,2,1,3,Returning_Visitor,False,False
6,0,0.0,0.0,0.0,1,0.000000,0.200000,0.200000,0.0,0.4,Feb,2.0,4,3,3,Returning_Visitor,False,False
7,1,0.0,0.0,0.0,0,0.000000,0.200000,0.200000,0.0,0.0,Feb,1.0,2,1,5,Returning_Visitor,True,False
8,0,0.0,0.0,0.0,2,37.000000,0.000000,0.100000,0.0,0.8,Feb,2.0,2,2,3,Returning_Visitor,False,False
9,0,0.0,0.0,0.0,3,738.000000,0.000000,0.022222,0.0,0.4,Feb,2.0,4,1,2,Returning_Visitor,False,False


In [230]:
# Examine dimensions of the dataset
rows, columns = df.shape

print('Rows:', rows, 'Columns:', columns)

# Examine variables and data types of variables
df.dtypes

Rows: 12330 Columns: 18


Administrative               int64
Administrative_Duration    float64
Informational              float64
Informational_Duration     float64
ProductRelated               int64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems           float64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
Revenue                       bool
dtype: object

In [232]:
# Examine values for categorical data
print(df['VisitorType'].unique())
print(df['Month'].unique())
print(df['Weekend'].unique())
print(df['Revenue'].unique())


['Returning_Visitor' 'New_Visitor' 'Other']
['Feb' 'Mar' 'May' 'Oct' 'June' 'Jul' 'Aug' 'Nov' 'Sep' 'Dec']
[False  True]
[False  True]


In [234]:
# Examine missing data
df.isnull().sum()

Administrative               0
Administrative_Duration      0
Informational              128
Informational_Duration       0
ProductRelated               0
ProductRelated_Duration      0
BounceRates                  0
ExitRates                    0
PageValues                 135
SpecialDay                   0
Month                        0
OperatingSystems           123
Browser                      0
Region                       0
TrafficType                  0
VisitorType                  0
Weekend                      0
Revenue                      0
dtype: int64

There are 128 instances of missing data for 'Informational', 135 instances of missing data for 'PageValues' and 123 instances of missing data for 'OperatingSystems'.

#### Handling Missing Data

Informational - Per the data notes, Informational represents the number of informational pages visited by the visitor and Informational_Duration represents the total time spent on informational pages. Therefore if  Informational_Duration is equal to zero, we impute Informational with the value 0. As this indicates the user spent zero time on any information pages. For missing values of Informational where Informational_Duration is greater than zero, we use the mean value of Informational based on the quartiles of Informational_Duration; The imputed value is proportional to the time spent on the pages.

PageValues - Per the data notes, this variable is from Google Analytics, it estimates the average value of a page based on the revenue generated from transactions that occur after a user visits that page. Therefore if Revenue equals False (no purchase), we impute PageValues with zero, as we assume that the pages visited did not contribute to a purchase thus their value is zero. For other cases, we impute missing values with the mean of PageValues where Revenue equals True.

OperatingSystems - Impute missing values with -999 to represent that the value is 'Unknown'. Using the mean would not be appropriate as Operating Systems is a nominal variable. Additionally, we do not plan to include Operating Systems in the final model, so imputation is not critical.


In [239]:
# Impute missing values of Informational

# Impute missing Informational values as 0 where Informational_duration is 0
df.loc[(df['Informational'].isna()) & (df['Informational_Duration'] == 0), 'Informational'] = 0

# Check missing values
df.isnull().sum()

# There are 24 missing values of Informational still.

Administrative               0
Administrative_Duration      0
Informational               24
Informational_Duration       0
ProductRelated               0
ProductRelated_Duration      0
BounceRates                  0
ExitRates                    0
PageValues                 135
SpecialDay                   0
Month                        0
OperatingSystems           123
Browser                      0
Region                       0
TrafficType                  0
VisitorType                  0
Weekend                      0
Revenue                      0
dtype: int64

In [241]:
# Impute the rest of the missing values of Informational based on the mean of Informational based on quartiles of Informational_duration

# Construct quartile boundaries
Q1 = df[df['Informational_Duration'] > 0]['Informational_Duration'].quantile(0.25)
Q2 = df[df['Informational_Duration'] > 0]['Informational_Duration'].quantile(0.50)
Q3 = df[df['Informational_Duration'] > 0]['Informational_Duration'].quantile(0.75)

print(Q1, Q2, Q3)

# Construct group means based of quartile boundries
mean_Q1 = df[(df['Informational_Duration'] > 0) & (df['Informational_Duration'] <= Q1)]['Informational'].mean()
mean_Q2 = df[(df['Informational_Duration'] > Q1) & (df['Informational_Duration'] <= Q2)]['Informational'].mean()
mean_Q3 = df[(df['Informational_Duration'] > Q2) & (df['Informational_Duration'] <= Q3)]['Informational'].mean()
mean_Q4 = df[(df['Informational_Duration'] > Q3)]['Informational'].mean()

print(mean_Q1, mean_Q2, mean_Q3, mean_Q4)

# Impute means based on the value of Informational_duration
df.loc[(df['Informational'].isna()) & (df['Informational_Duration'] <= Q1), 'Informational'] = mean_Q1
df.loc[(df['Informational'].isna()) & (df['Informational_Duration'] > Q1) & (df['Informational_Duration'] <= Q2), 'Informational'] = mean_Q2
df.loc[(df['Informational'].isna()) & (df['Informational_Duration'] > Q2) & (df['Informational_Duration'] <= Q3), 'Informational'] = mean_Q3
df.loc[(df['Informational'].isna()) & (df['Informational_Duration'] > Q3), 'Informational'] = mean_Q4

# Check missing values
df.isnull().sum()

28.0 76.0 201.4
1.5277310924369747 1.9142857142857144 2.6884422110552766 3.8097643097643097


Administrative               0
Administrative_Duration      0
Informational                0
Informational_Duration       0
ProductRelated               0
ProductRelated_Duration      0
BounceRates                  0
ExitRates                    0
PageValues                 135
SpecialDay                   0
Month                        0
OperatingSystems           123
Browser                      0
Region                       0
TrafficType                  0
VisitorType                  0
Weekend                      0
Revenue                      0
dtype: int64

In [243]:
# Impute missing values of PageValues

# Impute missing PageValues values as 0 where Revenu is False
df.loc[(df['PageValues'].isna()) & (df['Revenue'] == False), 'PageValues'] = 0

# Calculate mean PageValue for sessions that generated revenue
mean_revenue_value = df[df['Revenue'] == True]['PageValues'].mean()

# Impute missing values where Revenue is True
df.loc[(df['PageValues'].isna()) & (df['Revenue'] == True), 'PageValues'] = mean_revenue_value

# Check missing values
df.isnull().sum()

Administrative               0
Administrative_Duration      0
Informational                0
Informational_Duration       0
ProductRelated               0
ProductRelated_Duration      0
BounceRates                  0
ExitRates                    0
PageValues                   0
SpecialDay                   0
Month                        0
OperatingSystems           123
Browser                      0
Region                       0
TrafficType                  0
VisitorType                  0
Weekend                      0
Revenue                      0
dtype: int64

In [245]:
# Impute missing values of OperatingSystems

# Impute missing OperatingSystems as '-999' to represent Unknown
df.loc[df['OperatingSystems'].isna(), 'OperatingSystems'] = -999

# Check for any remaining missing values
df.isnull().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

#### Transform the data for easier interpretability


In [261]:
# Round duration variables to three decimal places
df[['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']] = df[['Administrative_Duration', 'Informational_Duration', 'ProductRelated_Duration']].round(3)

# Round rate variables to three decimal places
df[['BounceRates', 'ExitRates']] = df[['BounceRates', 'ExitRates']].round(3)

# Round Pagealues to two decimal places (to represent dollars and cents)
df['PageValues'] = df['PageValues'].round(2)


In [263]:
# Examine the transformed data
df.head(10)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0.0,0.0,1,0.000,0.200,0.200,0.0,0.0,Feb,1.0,1,1,1,Returning_Visitor,False,False
1,0,0.0,0.0,0.0,2,64.000,0.000,0.100,0.0,0.0,Feb,2.0,2,1,2,Returning_Visitor,False,False
2,0,0.0,0.0,0.0,1,0.000,0.200,0.200,0.0,0.0,Feb,4.0,1,9,3,Returning_Visitor,False,False
3,0,0.0,0.0,0.0,2,2.667,0.050,0.140,0.0,0.0,Feb,3.0,2,2,4,Returning_Visitor,False,False
4,0,0.0,0.0,0.0,10,627.500,0.020,0.050,0.0,0.0,Feb,3.0,3,1,4,Returning_Visitor,True,False
5,0,0.0,0.0,0.0,19,154.217,0.016,0.025,0.0,0.0,Feb,2.0,2,1,3,Returning_Visitor,False,False
6,0,0.0,0.0,0.0,1,0.000,0.200,0.200,0.0,0.4,Feb,2.0,4,3,3,Returning_Visitor,False,False
7,1,0.0,0.0,0.0,0,0.000,0.200,0.200,0.0,0.0,Feb,1.0,2,1,5,Returning_Visitor,True,False
8,0,0.0,0.0,0.0,2,37.000,0.000,0.100,0.0,0.8,Feb,2.0,2,2,3,Returning_Visitor,False,False
9,0,0.0,0.0,0.0,3,738.000,0.000,0.022,0.0,0.4,Feb,2.0,4,1,2,Returning_Visitor,False,False


#### Transform/Construct additonal variables for modeling

In [269]:
# Create a variable that is the total duration across all page types;
# Create a variable that is the total number of pages visited

df['Total_Duration'] = df['Administrative_Duration'] + df['Informational_Duration'] + df['ProductRelated_Duration'] 
df['Total_Pages'] = df['Administrative'] + df['Informational'] + df['ProductRelated'] 


#### Discretization - TO BE FINALIZED LATER
We use the Pandas function 'qcut' to sort duration variables into three bins with equal number of observations. Setting q=3 will sort the data and split it into three equal sized groups. The low category contains the bottom third of durations, the medium category contains the middle third durations, and the high category contains the top third. 
Reference: https://www.geeksforgeeks.org/discretization/


In [283]:
# Examine the number of zeros in each cateogry
print((df['ProductRelated_Duration'] == 0).sum())
print((df['Administrative_Duration'] == 0).sum())
print((df['Informational_Duration'] == 0).sum())
print((df['Total_Duration'] == 0).sum())


755
5903
9925
720


In [281]:
# Discretize Duration variables into 3  bins 
# The duplicates=drop is used with variables that have a large amount of zeros and cannot create three bins.
df['ProductRelated_Duration_bins'] = pd.qcut(df['ProductRelated_Duration'], q=3, labels=['Low', 'Medium', 'High'])
df['Administrative_Duration_bins'] = pd.qcut(df['Administrative_Duration'], q=3, labels=['Low', 'High'], duplicates='drop')
df['Informational_Duration_bins'] = pd.qcut(df['Informational_Duration'], q=3, labels=['High'], duplicates='drop')
df['Total_Duration_bins'] = pd.qcut(df['Total_Duration'], q=3, labels=['Low', 'Medium', 'High'])

# Check bin distribution
print(df['ProductDuration_bins'].value_counts())
print(df['Administrative_Duration_bins'].value_counts())
print(df['Informational_Duration_bins'].value_counts())
print(df['Total_Duration_bins'].value_counts())



ProductDuration_bins
Low       4110
Medium    4110
High      4110
Name: count, dtype: int64
Administrative_Duration_bins
Low     8223
High    4107
Name: count, dtype: int64
Informational_Duration_bins
High    12330
Name: count, dtype: int64
Total_Duration_bins
Low       4110
Medium    4110
High      4110
Name: count, dtype: int64


## 3/26/25 NOTE:
Depending on the final model we decide on, we can subset the dataset the reduce any redundant data;
I believe variables that we end up dropping will be operating systems, Browser.. Also due to the large amount of zero under Administrative and Informational potentially we should just utilize Product Related/Product duration

